In [ ]:
!pip install h5py
!pip install torch-interpol
!pip uninstall numpy -y
!pip install numpy==1.22
!pip install cupy-cuda113
!pip install sigpy
!pip install matplotlib
pip install cupy-cuda11x
!pip install imageio==1.3
#!pip install cupy-cuda113

In [ ]:
import numpy as np
import cupy
import sigpy as sp
bpt=np.load('bpt.npy','r+')
mps=np.load('mps0.npy','r+')
ksp=np.load('kspa0.npy','r+')
coord=np.load('coorda0.npy','r+')
dcf=np.load('dcfa0.npy','r+')


In [ ]:
from bpt_map import compute_bpt_over_frames,norm_data,run_unroll1,adj_field_solver
from gated_data_loader import init_im,flows, warp1
from moco_scripts14 import convert_utea,pca_cc,autofova,gen_template,for_field_solver,inv_field_solver,inv_field_solver_refine,gen_MSLR_for,gen_MSLR_adj,gen,flows,save_images_inv,f,save_images_for,rescale_flow,_updateb,warp1,MoCo_MSLR
import torch

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
frames=500 #num frames you wish to reconstruct
channels=31 #num coils
RO=123 #spatial resolution
spokes_per_bin=ksp.shape[1]//frames #sets temporal resolution
bpt_norm=compute_bpt_over_frames(frames, channels,bpt,ksp) #normed bpt
mps, kspa,coorda,dcfa=norm_data(ksp,coord,dcf,RO,spokes_per_bin) #normed acquisition data
im=init_im(torch.from_numpy(kspa).cuda(),torch.from_numpy(dcfa).cuda(),torch.from_numpy(coorda).cuda(),torch.from_numpy(mps).cpu()) #generate initial template image
#conjugate gradient descent (iters=num of iterations, alpha is we
alpha=torch.zeros([1]).cuda()
iters=10
im_template=run_unroll1(im,torch.from_numpy(kspa).cuda(),torch.from_numpy(coorda).cuda(),torch.from_numpy(dcfa).cuda(),torch.from_numpy(mps).cuda(),alpha,iters) #data_consistent template image with CG steps

#goal of CG descent is to take the template image and when you transform it to k-space that this approximation
#of the k-space data is approximately scaled the same as the acquired k-space data

In [ ]:

def calculate_sense0(M_t,ksp,mps_c,coord_t,dcf):
        F = sp.linop.NUFFT([mps_c.shape[0], mps_c.shape[1], mps_c.shape[2]], coord_t.detach().cpu().numpy(),1.25,2)
        F_torch = sp.to_pytorch_function(F, input_iscomplex=True, output_iscomplex=True)
        FH_torch = sp.to_pytorch_function(F.H, input_iscomplex=True, output_iscomplex=True)
        e_tc=F_torch.apply(M_t.cuda())
        e_tca=torch.complex(e_tc[:,0],e_tc[:,1]) 
        print(torch.abs(ksp).max())
        print(torch.abs(e_tca).max())
        res=(ksp-e_tca)*(dcf)**0.5 
        res=torch.reshape(res,[1,-1])
        lossb=(torch.linalg.norm(res,2))**2 
        return lossb

def _updateb(img_t,ksp_t,dcf_t,coord_t,mpsa): #ima,deform_adjoint1,ksp,coord,dcf,mps,t,device,tr_per_frame):
  
# Data consistency.
  loss_t=0
  for c in range(mpsa.shape[0]):
       
    
    loss_t=loss_t+torch.utils.checkpoint.checkpoint(calculate_sense0,torch.cat([torch.reshape(torch.real(img_t*mpsa[c].cuda()),[mpsa.shape[1],mpsa.shape[2],mpsa.shape[3],1]),torch.reshape(torch.imag(img_t*mpsa[c].cuda()),[mpsa.shape[1],mpsa.shape[2],mpsa.shape[3],1])],axis=3),torch.reshape(ksp_t[c],[-1]),mpsa[c],torch.reshape(coord_t,[-1,3]),torch.reshape(dcf_t,[-1]))
   
  
  return loss_t

In [ ]:
from gated_data_loader import gen_MSLR #initializes the  MSLR motion fields
T=500
rank=1
block_size_for=[8,16,32]
block_size_adj=block_size_for
scale=1
deformL_param_adj0,deformR_param_adj0,deformL_param_for0,deformR_param_for0,block_torch0,ishape0a=gen_MSLR(T,rank,block_size_adj,block_size_for,scale,mps) #initializes the motion fields



In [ ]:
#this solves for the MSLR components of the motion fields, goal here is to solve for
#estimate of L deformation field spatial basis
T=50
iter_adj=50 #sets the number of iterations
RO=123
init_frame=400
final_frame=450
weight_MSE=20
weight_smoother=10
adj_field_solver(deformL_param_adj0,deformR_param_adj0,deformL_param_for0,deformR_param_for0,im_template.detach().cpu().numpy(),ksp,coord,dcf,mps,iter_adj,RO,block_torch0,ishape0a,init_frame,final_frame,1,mps,spokes_per_bin,weight_MSE,weight_smoother)




In [ ]:
for i in range(3):
    np.save('deformL_param_adj0s'+str(i),deformL_param_adj0[i].detach().cpu().numpy())
    np.save('deformR_param_adj0s'+str(i),deformR_param_adj0[i].detach().cpu().numpy())

In [ ]:
R_blocks=deformR_param_adj0

In [ ]:
#fully connected neural network with a bunch of hard coded parameters, this could be replaced by GP or CNN

import torch.nn as nn
class NetRresp(nn.Module):
    def __init__(self):
        super(NetRresp, self).__init__()
       # self.conv1b=nn.Conv2d(2,16,[7,1],padding=3,bias=False)
       # self.conv2b=nn.Conv2d(16,2,[1,1],padding=0,bias=False)
        
        self.a0=torch.nn.Linear(31, 3*31*10, bias=False, device=None, dtype=None)
      
        self.a1=torch.nn.Linear(3*31*10,3*31*10*3, bias=False, device=None, dtype=None)
        self.a2=torch.nn.Linear(3*31*10*3,31*10*3, bias=False, device=None, dtype=None)
        self.a3=torch.nn.Linear(31*10*3,31*10*3, bias=False, device=None, dtype=None)
        self.a4=torch.nn.Linear(31*10*3, 31*10*3, bias=False, device=None, dtype=None)
     #   self.R0=torch.nn.Linear(100,8*3, bias=False, device=None, dtype=None)
      #  self.R0y=torch.nn.Linear(20,8, bias=False, device=None, dtype=None)
      #  self.R0z=torch.nn.Linear(20,8, bias=False, device=None, dtype=None)
        self.R0=torch.nn.Linear(31*10*3, R_blocks[0].shape[1]*R_blocks[0].shape[2]*3, bias=False, device=None, dtype=None)
        self.R1=torch.nn.Linear(31*10*3, R_blocks[1].shape[1]*R_blocks[1].shape[2]*3, bias=False, device=None, dtype=None)
        self.R2=torch.nn.Linear(31*10*3,R_blocks[2].shape[1]*R_blocks[2].shape[2]*3, bias=False, device=None, dtype=None)
        self.a5x=torch.nn.Linear(1000, 8, bias=False, device=None, dtype=None)
        self.a6x=torch.nn.Linear(8, 1000, bias=False, device=None, dtype=None)
        self.a7x=torch.nn.Linear(1000, 1000, bias=False, device=None, dtype=None)
        self.R0x=torch.nn.Linear(1000, R_blocks[0].shape[1]*R_blocks[0].shape[2], bias=False, device=None, dtype=None)
        self.R1x=torch.nn.Linear(1000, R_blocks[1].shape[1]*R_blocks[1].shape[2], bias=False, device=None, dtype=None)
        self.R2x=torch.nn.Linear(1000,R_blocks[2].shape[1]*R_blocks[2].shape[2], bias=False, device=None, dtype=None)

        self.a5y=torch.nn.Linear(1000, 8, bias=False, device=None, dtype=None)
        self.a6y=torch.nn.Linear(8, 1000, bias=False, device=None, dtype=None)
        self.a7y=torch.nn.Linear(1000, 1000, bias=False, device=None, dtype=None)
        self.R0y=torch.nn.Linear(1000, R_blocks[0].shape[1]*R_blocks[0].shape[2], bias=False, device=None, dtype=None)
        self.R1y=torch.nn.Linear(1000, R_blocks[1].shape[1]*R_blocks[1].shape[2], bias=False, device=None, dtype=None)
        self.R2y=torch.nn.Linear(1000,R_blocks[2].shape[1]*R_blocks[2].shape[2], bias=False, device=None, dtype=None)

        self.a5z=torch.nn.Linear(1000, 8, bias=False, device=None, dtype=None)
        self.a6z=torch.nn.Linear(8, 1000, bias=False, device=None, dtype=None)
        self.a7z=torch.nn.Linear(1000, 1000, bias=False, device=None, dtype=None)
        self.R0z=torch.nn.Linear(1000, R_blocks[0].shape[1]*R_blocks[0].shape[2], bias=False, device=None, dtype=None)
        self.R1z=torch.nn.Linear(1000, R_blocks[1].shape[1]*R_blocks[1].shape[2], bias=False, device=None, dtype=None)
        self.R2z=torch.nn.Linear(1000,R_blocks[2].shape[1]*R_blocks[2].shape[2], bias=False, device=None, dtype=None)
       # self.Gx
       # self.Gy
       # self.Gz

     
       
        self.activation=torch.nn.ReLU(inplace=True)
        
    def forward(self, x):
        x=self.a0(x)
        x=self.activation(x)
        '''
        x=self.a0a(x)
        x=self.activation(x)
        x=self.a0b(x)
        T=self.activation(x)
        '''
        x=self.a1(x)
        x=self.activation(x)
        x=self.a2(x)
        x=self.activation(x)
        x=self.a3(x)
        x=self.activation(x)
        x=self.a4(x)
        x=self.activation(x)
        Ra=torch.reshape(self.R0(x),[3,1760,1,1])
        Rb=torch.reshape(self.R1(x),[3,200,1,1])
        Rc=torch.reshape(self.R2(x),[3,16,1,1])
        Rresp=[Ra,Rb,Rc]
        '''
        x=self.a5x(x)
        Tx=self.activation(x)
        x=self.a6x(Tx)
        x=self.activation(x)
        x=self.a7x(x)
        Rax=torch.reshape(self.R0x(x),[1,1760,1,1])
        Rbx=torch.reshape(self.R1x(x),[1,200,1,1])
        Rcx=torch.reshape(self.R2x(x),[1,16,1,1])

        x=self.a5y(x)
        Ty=self.activation(x)
        x=self.a6y(Tx)
        x=self.activation(x)
        x=self.a7y(x)
        Ray=torch.reshape(self.R0y(x),[1,1760,1,1])
        Rby=torch.reshape(self.R1y(x),[1,200,1,1])
        Rcy=torch.reshape(self.R2y(x),[1,16,1,1])

        x=self.a5z(x)
        Tz=self.activation(x)
        x=self.a6z(Tx)
        x=self.activation(x)
        x=self.a7z(x)
        Raz=torch.reshape(self.R0z(x),[1,1760,1,1])
        Rbz=torch.reshape(self.R1z(x),[1,200,1,1])
        Rcz=torch.reshape(self.R2z(x),[1,16,1,1])
        Rresp=[Ra,Rb,Rc]
        Rrespx=[Rax,Rbx,Rcx]
        Rrespy=[Ray,Rby,Rcz]
        Rrespz=[Raz,Rbz,Rcz]
       # t0x=torch.reshape(t0x,[3,8,1])
       # t0y=self.R0y(x)
       # t0y=torch.reshape(t0y,[1,8,1])
       # t0z=self.R0z(x)
       # t0z=torch.reshape(t0z,[1,8,1])
       # t0=torch.cat([t0x,t0y,t0z],axis=0)
       # t1=self.R1(x)
       # t2=self.R2(x)
        '''
        return Rresp #,Rrespx,Rrespy,Rrespz,Tx,Ty,Tz

netresp=NetRresp()
netresp=netresp.cuda() #puts the neural network on the GPU

In [ ]:
#initialize your optimizer
optimizer0 = torch.optim.Adam(
    netresp.parameters(),
    lr= 1e-3
)
all0=torch.abs(ima).cuda().max()
image_look=np.zeros([50,mps.shape[2],mps.shape[3]])

In [ ]:

def calculate_sense0(M_t,ksp,mps_c,coord_t,dcf):
        torch.cuda.empty_cache()
        loss=0
      
        F = sp.linop.NUFFT([mps_c.shape[0], mps_c.shape[1], mps_c.shape[2]], coord_t.detach().cpu().numpy(),1.25,2)
        F_torch = sp.to_pytorch_function(F, input_iscomplex=True, output_iscomplex=True)
        FH_torch = sp.to_pytorch_function(F.H, input_iscomplex=True, output_iscomplex=True)
        e_tc=F_torch.apply(M_t.cuda())
      
        e_tca=torch.complex(e_tc[:,0],e_tc[:,1]) #*torch.reshape(Ptc,[-1])
        #print(torch.abs(e_tca).max())
        #print(torch.abs(ksp).max())
        torch.cuda.empty_cache()
       
        res=(e_tca-ksp)*(dcf)**0.5 #**0.5 #**0.5
        res=torch.reshape(res,[1,-1])
        lossb=(torch.linalg.norm(res,2))**2 #**2 #**2 #torch.abs(torch.sum(((ksp-e_tca))))
        
        #loss=(torch.norm(resk)) #/torch.norm(ksp,2)+torch.norm(resk,1)/torch.norm(ksp,1) #torch.abs(torch.sum((e_tca.cuda()*dcf.cuda()**0.5-ksp_t.cuda()*dcf.cuda()**0.5)**2))
       
        
       # torch.cuda.empty_cache()
       # loss=torch.norm(resk,1) #*index_all/index_max  #*index_all/ksp.shape[0]
        #print(loss)
        return lossb

def _updateb(img_t,ksp_t,dcf_t,coord_t,mpsa): #ima,deform_adjoint1,ksp,coord,dcf,mps,t,device,tr_per_frame):
  
# Data consistency.
  loss_t=0
  #print(mpsa.shape)
  #print(img_t.shape)
  #n=int(cooo[0])
  for c in range(mpsa.shape[0]):
       
    
    loss_t=loss_t+torch.utils.checkpoint.checkpoint(calculate_sense0,torch.cat([torch.reshape(torch.real(img_t*mpsa[c].cuda()),[mpsa.shape[1],mpsa.shape[2],mpsa.shape[3],1]),torch.reshape(torch.imag(img_t*mpsa[c].cuda()),[mpsa.shape[1],mpsa.shape[2],mpsa.shape[3],1])],axis=3),torch.reshape(ksp_t[c],[-1]),mpsa[c],torch.reshape(coord_t,[-1,3]),torch.reshape(dcf_t,[-1]))
   # m=mpsa[c].detach().cpu().numpy()
   # torch.cuda.empty_cache()
  #del img_t
  #torch.cuda.empty_cache()
  
  return loss_t

In [ ]:
iter_network=100

In [ ]:
#this is how you train the network
for i in range(iter_network):
    T1=400
    T2=450
   
   # respa.requires_grad=True
   # ecga.requires_grad=True
    lossa=0
  
    K=np.random.randint(T1,T2,50)
    a=0
    for j in K: #range(450,500):
        
    
        Rresp=netresp(torch.abs(torch.reshape(torch.from_numpy(bpt_norm[j:j+1,:31]).cuda(),[-1]))) #network ouputs estimate of temporal basis
      
        loss_tot=torch.nn.MSELoss()
        deforma=flows(deformL_param_adj0,Rresp,0,block_torch0,ishape0a) #flows is a function that combined the compressed representation into a motion field at time t
     
        im_out=warp1((im_temp).cuda()/norm0,deforma,mps,complex=True) #warp1 is it warps the template image according to deforma
      
       
        tr_start=spokes_per_bin*j
        tr_end=spokes_per_bin*(j+1)
        ksp_ta=torch.from_numpy(ksp[:,tr_start:tr_end,:]).cuda()/norm0
        coord_t=torch.from_numpy(coord[tr_start:tr_end,:]).cuda()
        dcf_t=torch.from_numpy(dcf[tr_start:tr_end,:]).cuda()
        loss_grad=f.loss(deforma) #smooth the motion field
        index_max=1
        index_frame=1
    
        loss_for=_updateb(im_out.unsqueeze(0),ksp_ta,dcf_t,coord_t,torch.from_numpy(mps).cpu())*1+loss_grad*1e-1 #+loss_ortho*1e-4+loss_grad #*20+loss_grad*10 #minimizing/enforcing the loss in k-space between the warped template image (im_out and the original k-space data at time point t)
        loss_for.backward() #backprop gradients
   
        lossa=lossa+loss_for
      
        im_out=torch.squeeze(im_out)
       
        image_look[j-400]=np.abs(im_out[20].detach().cpu().numpy())
        a=a+1
        
       
  

     
  
        optimizer0.step()
        optimizer0.zero_grad()
        optimizer1.step()
        optimizer1.zero_grad()
    print(lossa) #loss over all frames
    import imageio
    imageio.mimsave('image.gif', [np.abs(image_look[i,:,:])*1e15 for i in range(50)], fps=5)
